In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
import numpy as np

warnings.filterwarnings("ignore")
sns.set(style="whitegrid", color_codes=True)
%matplotlib inline

# SALES INPUT

In [2]:
data_sales = pd.read_excel("Training_Data.xlsx")
print(data_sales.shape)
data_sales.head()

(780829, 9)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,3,2014-12-31,2605,327,1,0,0,1
1,2,3,2014-12-31,2269,252,1,0,0,1
2,3,3,2014-12-31,3804,408,1,0,0,1
3,4,3,2014-12-31,10152,1311,1,0,0,1
4,5,3,2014-12-31,1830,217,1,0,0,1


In [3]:
validratio = ~data_sales.isnull()
validratio = validratio.sum(axis=0).iloc[::-1]
validratio

SchoolHoliday    780829
StateHoliday     780829
Promo            780829
Open             780829
Customers        780829
Sales            780829
Date             780829
DayOfWeek        780829
Store            780829
dtype: int64

# STORES INPUT

In [4]:
data_stores = pd.read_csv("stores.csv")
print(data_stores.shape)
data_stores.head()

(1115, 10)


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [5]:
validratio = ~data_stores.isnull()
validratio = validratio.sum(axis=0).iloc[::-1]
validratio = pd.DataFrame(validratio/data_stores.shape[0] * 100, columns=['Percentage of valid values'])
validratio

,Percentage of valid values
PromoInterval,51.210762
Promo2SinceYear,51.210762
Promo2SinceWeek,51.210762
Promo2,100.000000
CompetitionOpenSinceYear,68.251121
CompetitionOpenSinceMonth,68.251121
CompetitionDistance,99.730942
Assortment,100.000000
StoreType,100.000000
Store,100.000000


# FILL VALUES

In [6]:
cd_mean = data_stores['CompetitionDistance'].mean()
data_stores['CompetitionDistance'] = data_stores['CompetitionDistance'].fillna(cd_mean)
cd_mean

5404.901079136691

# JOIN DATA

In [7]:
data_train = data_sales.join(data_stores, 'Store', rsuffix='_OL', how='inner')
print(data_train.shape)
data_train.head()

(780099, 19)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Store_OL,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,3,2014-12-31,2605,327,1,0,0,1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
935,1,2,2014-12-30,6466,703,1,0,0,1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
1870,1,1,2014-12-29,6463,700,1,0,0,1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2805,1,7,2014-12-28,0,0,0,0,0,1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
3740,1,6,2014-12-27,6057,684,1,0,0,1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"


In [8]:
data_train = data_train.drop(['Store_OL'], axis=1)

# FEATURES CLEANING AND ENGINEERING

In [9]:
data_train['Day'] = data_train['Date'].dt.day
data_train['Month'] = data_train['Date'].dt.month
data_train['Year'] = data_train['Date'].dt.month
data_train = data_train.drop(['DayOfWeek'], axis=1)

In [10]:
# CompetitionOpenSince___ has irrelavent data
data_train = data_train.drop(['CompetitionOpenSinceYear', 'CompetitionOpenSinceMonth'], axis=1)

In [11]:
import datetime
from datetime import timedelta

def promo3(x):
    if x['Promo2'] == 0:
        return False
    if pd.isna(x['Promo2SinceYear']) or pd.isna(x['PromoInterval']):
        return False
    
    promoDt = datetime.date(int(x['Promo2SinceYear']), 1, 1)
    if not pd.isna(x['Promo2SinceWeek']):
        promoDt += timedelta(weeks=x['Promo2SinceWeek'])
    
    if promoDt <= x['Date'].date():
        return x['Date'].strftime('%b') in x['PromoInterval']
    else:
        return False

data_train['OnRecurringPromo'] = data_train.apply(promo3, axis=1)

In [12]:
data_train = data_train.drop(['Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval'], axis=1)

In [13]:
data_train.head()

,Store,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,Day,Month,Year,OnRecurringPromo
0,1,2014-12-31,2605,327,1,0,0,1,a,a,570.0,1,31,12,12,False
935,1,2014-12-30,6466,703,1,0,0,1,a,a,570.0,1,30,12,12,False
1870,1,2014-12-29,6463,700,1,0,0,1,a,a,570.0,1,29,12,12,False
2805,1,2014-12-28,0,0,0,0,0,1,a,a,570.0,1,28,12,12,False
3740,1,2014-12-27,6057,684,1,0,0,1,a,a,570.0,1,27,12,12,False


# Normalize Values

In [14]:
data_train['Customers_n'] = (data_train['Customers'] - data_train['Customers'].mean())/data_train['Customers'].std()
data_train['CompetitionDistance_n'] = (data_train['CompetitionDistance'] - data_train['CompetitionDistance'].mean())/data_train['CompetitionDistance'].std()

In [15]:
data_train.head()

,Store,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,Day,Month,Year,OnRecurringPromo,Customers_n,CompetitionDistance_n
0,1,2014-12-31,2605,327,1,0,0,1,a,a,570.0,1,31,12,12,False,-0.660630,-0.631116
935,1,2014-12-30,6466,703,1,0,0,1,a,a,570.0,1,30,12,12,False,0.142926,-0.631116
1870,1,2014-12-29,6463,700,1,0,0,1,a,a,570.0,1,29,12,12,False,0.136514,-0.631116
2805,1,2014-12-28,0,0,0,0,0,1,a,a,570.0,1,28,12,12,False,-1.359468,-0.631116
3740,1,2014-12-27,6057,684,1,0,0,1,a,a,570.0,1,27,12,12,False,0.102321,-0.631116


# Save as trainable format

In [16]:
inputSet = np.zeros((data_train.shape[0], 19), dtype=np.float32)
outputSet = np.zeros(data_train.shape[0], dtype=np.float32)
#idxToId = np.zeros(data_train.shape[0], dtype=np.int32)
for idx, x in enumerate(data_train.iterrows()):
    row = x[1]
    
    #idxToId[idx] = row['PassengerId']
    outputSet[idx] = row['Sales']
        
    inputSet[idx][0] = row['Customers_n']
    inputSet[idx][1] = row['Open']
    
    if row['StateHoliday'] == 'a': inputSet[idx][2] = 1
    elif row['StateHoliday'] == 'b': inputSet[idx][3] = 1
    elif row['StateHoliday'] == 'c': inputSet[idx][4] = 1
        
    inputSet[idx][5] = row['SchoolHoliday']
    
    if row['StoreType'] == 'a': inputSet[idx][6] = 1
    elif row['StoreType'] == 'b': inputSet[idx][7] = 1
    elif row['StoreType'] == 'c': inputSet[idx][8] = 1
    elif row['StoreType'] == 'd': inputSet[idx][9] = 1
    
    if row['Assortment'] == 'a': inputSet[idx][10] = 1
    elif row['Assortment'] == 'b': inputSet[idx][11] = 1
    elif row['Assortment'] == 'c': inputSet[idx][12] = 1
        
    inputSet[idx][13] = row['Promo']
    inputSet[idx][14] = row['OnRecurringPromo']
    inputSet[idx][15] = row['Day']/31
    inputSet[idx][16] = row['Month']/12
    inputSet[idx][17] = row['CompetitionDistance_n']

In [17]:
np.savez("dataset.npz", inputSet=inputSet, outputSet=outputSet)

In [18]:
np.sum(inputSet, axis=0).astype(np.int32)

array([    11, 647754,  14749,   4456,   4096, 146271, 423635,  12042,
       101422, 243000, 415231,   6386, 358482, 293030, 109082, 395470,
       416071,      0,      0], dtype=int32)